## Investigate effect of object detections on ADL detection

Check if detections have an impact on the performance of ADL detection by comparing the performance of models trained on detected active objects versus labelled active objects.

In [1]:
import os
from collections import Counter

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier

import egoviz.models.evaluation as ev
import egoviz.models.processing as pr

SEED = 42

pd.set_option("display.max_rows", 300)

In [2]:
cwd = os.getcwd()
file_path = os.path.join(cwd, '../data/home_data_eval_all_preds.pkl')
data = pr.load_pickle(file_path)

In [3]:
df_dets = pd.DataFrame(columns=['video', 'frame', 'classes', 'active', 'adl'])

for id, dets in data.items():
    adl = id.split('_', 1)[0]
    video = id.split('_')[1]
    frame = id.split('_')[2]
    classes = dets['detic_data']['labels']
    active = dets['detic_data']['active']

    row = {'video': video, 'frame': frame, 'classes': classes, 'adl': adl, 'active': active}

    df_dets.loc[len(df_dets)] = row

df_dets.head()

,video,frame,classes,active,adl
0,SCI06-7--11,frame49,"[phone_tablet, electronics, furniture, furnitu...","[True, False, False, False, False, False, Fals...",communication-mgmt
1,SCI06-7--11,frame98,"[phone_tablet, furniture, furniture, cabinetry...","[True, False, False, False, False, False, Fals...",communication-mgmt
2,SCI06-7--11,frame196,"[phone_tablet, cabinetry, furniture, furniture...","[True, False, False, False, False, False, Fals...",communication-mgmt
3,SCI06-7--11,frame147,"[phone_tablet, furniture, cabinetry, cabinetry...","[True, False, False, False, False, False, Fals...",communication-mgmt
4,SCI06-7--11,frame294,"[phone_tablet, furniture, kitchen_appliance, f...","[True, False, False, False, False, False, Fals...",communication-mgmt


In [4]:
df_truth = pd.DataFrame(columns=['video', 'frame', 'classes', 'active', 'adl'])

for id, dets in data.items():
    adl = id.split('_', 1)[0]
    video = id.split('_')[1]
    frame = id.split('_')[2]
    classes = dets['ground_truth']['labels']
    active = dets['ground_truth']['active']

    row = {'video': video, 'frame': frame, 'classes': classes, 'adl': adl, 'active': active}

    df_truth.loc[len(df_truth)] = row

df_truth.head()

,video,frame,classes,active,adl
0,SCI06-7--11,frame49,"[phone_tablet, furniture, furniture, electronics]","[True, False, False, False]",communication-mgmt
1,SCI06-7--11,frame98,"[phone_tablet, furniture, furniture, electroni...","[True, False, False, False, False]",communication-mgmt
2,SCI06-7--11,frame196,"[food, phone_tablet, furniture, furniture]","[False, True, False, False]",communication-mgmt
3,SCI06-7--11,frame147,"[phone_tablet, furniture, furniture, food]","[True, False, False, False]",communication-mgmt
4,SCI06-7--11,frame294,"[food, phone_tablet, furniture, furniture]","[False, True, False, False]",communication-mgmt


In [5]:
def count_occurrences(classes, active):
    class_counts = Counter(classes)
    active_counts = Counter({cls: sum([act and (cls == c) for act, c in zip(active, classes)]) for cls in set(classes)})
    return class_counts, active_counts

# Apply the function to create new columns
df_dets['class_counts'], df_dets['active_counts'] = zip(*df_dets.apply(lambda row: count_occurrences(row['classes'], row['active']), axis=1))
df_truth['class_counts'], df_truth['active_counts'] = zip(*df_truth.apply(lambda row: count_occurrences(row['classes'], row['active']), axis=1))

In [6]:
def process_df(df):
    # Create a new DataFrame from class_counts and active_counts
    counts_df = pd.DataFrame(df.apply(lambda row: {'adl': row['adl'], 'video': row['video'], **{f'count_{key}': value for key, value in row['class_counts'].items()}, **{f'active_{key}': value for key, value in row['active_counts'].items()}}, axis=1).tolist())
    # Group by video and sum the values for each video
    grouped_counts_df = counts_df.groupby('video').agg({**{'adl': 'first'}, **{col: 'sum' for col in counts_df.columns if col not in ['adl', 'video']}})

    return grouped_counts_df.reset_index()

In [7]:
df_truth_processed = process_df(df_truth)
df_dets_processed = process_df(df_dets)

dftruth_scaled = pr.row_wise_min_max_scaling(df_truth_processed)
dfdets_scaled = pr.row_wise_min_max_scaling(df_dets_processed)

In [18]:
dftruth_scaled.shape

(114, 56)

### Modeling

In [9]:
label_encoder = LabelEncoder()

In [10]:
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=SEED, class_weight='balanced')),
    ('Random Forest', RandomForestClassifier(random_state=SEED, class_weight='balanced')),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=SEED)),
    ('XGBoost', XGBClassifier(random_state=SEED)),
    ('SVM', SVC(random_state=SEED, class_weight='balanced')),
    ('MLP', MLPClassifier(random_state=SEED, learning_rate='adaptive', max_iter=1000, early_stopping=True))
]

results_truth = ev.evaluate_models(models, dftruth_scaled, label_encoder)

2023-12-03 22:31:35,109 - root - INFO - LOGOCV complete for LogisticRegression
2023-12-03 22:31:36,165 - root - INFO - LOGOCV complete for RandomForestClassifier
2023-12-03 22:31:42,578 - root - INFO - LOGOCV complete for GradientBoostingClassifier
2023-12-03 22:31:45,078 - root - INFO - LOGOCV complete for XGBClassifier
2023-12-03 22:31:45,203 - root - INFO - LOGOCV complete for SVC
2023-12-03 22:31:45,392 - root - INFO - LOGOCV complete for MLPClassifier


In [11]:
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=SEED, class_weight='balanced')),
    ('Random Forest', RandomForestClassifier(random_state=SEED, class_weight='balanced')),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=SEED)),
    ('XGBoost', XGBClassifier(random_state=SEED)),
    ('SVM', SVC(random_state=SEED, class_weight='balanced')),
    ('MLP', MLPClassifier(random_state=SEED, learning_rate='adaptive', max_iter=1000, early_stopping=True))
]

results_dets = ev.evaluate_models(models, dfdets_scaled, label_encoder)

2023-12-03 22:31:45,525 - root - INFO - LOGOCV complete for LogisticRegression
2023-12-03 22:31:46,664 - root - INFO - LOGOCV complete for RandomForestClassifier
2023-12-03 22:31:58,028 - root - INFO - LOGOCV complete for GradientBoostingClassifier
2023-12-03 22:32:00,399 - root - INFO - LOGOCV complete for XGBClassifier
2023-12-03 22:32:00,527 - root - INFO - LOGOCV complete for SVC
2023-12-03 22:32:00,714 - root - INFO - LOGOCV complete for MLPClassifier


In [14]:
results_dets[1][['median_precision', 'median_recall', 'median_f1', 'model']].groupby('model').first().reset_index()

,model,median_precision,median_recall,median_f1
0,GradientBoostingClassifier,0.516667,0.585714,0.456349
1,LogisticRegression,0.591667,0.500000,0.436111
2,MLPClassifier,0.694444,0.309524,0.166667
3,RandomForestClassifier,0.711111,0.683333,0.466667
4,SVC,0.600000,0.500000,0.500000
5,XGBClassifier,0.585714,0.479167,0.562500


In [15]:
results_truth[1][['median_precision', 'median_recall', 'median_f1', 'model']].groupby('model').first().reset_index()

,model,median_precision,median_recall,median_f1
0,GradientBoostingClassifier,0.653571,0.585714,0.408466
1,LogisticRegression,0.725000,0.600000,0.472222
2,MLPClassifier,0.612500,0.400000,0.333333
3,RandomForestClassifier,0.700000,0.600000,0.458333
4,SVC,0.700000,0.600000,0.416667
5,XGBClassifier,0.697222,0.600000,0.500000


### Conclusions

The ADL classification model performs slightly better on the detection data vs. the ground truth data. This could be due to the fact that the detection data on average contains more objects than the ground truth data where only objects related to the activity are annotated, therefore, the ADL classification model is able to leverage the contextual information from the detection of additional objects to make better predictions.